# Inspect and summarize SLURM benchmarks

## Overview

## Load, merge and compare benchmark summary `csv`s

We need a function to compare our SLURM summary output files and highlight differences in a table.

In [1]:
library(dplyr)
library(tidyr)

read_and_summarize_differences <- function(directory) {
  # List all CSV files in the directory
  files <- list.files(directory, pattern = "\\.csv$", full.names = TRUE, recursive = TRUE)
  files <- files[grepl("alpha_05", files)]
  #files <- files[!grepl("1000000", files)]
  
  # Read each file and store in a list
  data_list <- lapply(files, function(file) {
    data <- read.csv(file)
    # Add a column to identify the source file
    data$Source <- basename(file)
    return(data)
  })
  
  # Combine all data frames into one
  combined_data <- bind_rows(data_list)
  
  # Spread the data to wide format for comparison
  wide_data <- spread(combined_data, key = Parameter, value = Value)
  
  # Identify rows with at least one difference in values
  differing_rows <- wide_data %>%
    summarise(across(-Source, ~n_distinct(.) > 1)) %>%
    select(which(colSums(.) > 0)) %>%
    names()
  
  # Filter the combined data to include only differing parameters
  summary_data <- combined_data %>%
    filter(Parameter %in% differing_rows) %>%
    select(Parameter, Source, Value)
  
  # Pivot wider for a clearer summary table, if necessary
  summary_table <- summary_data %>%
    pivot_wider(names_from = Source, values_from = Value)
  
  return(summary_table)
}


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
directory <- "/dcs04/lieber/statsgen/mnagle/mwas/CpGWAS/output/"
summary_table <- read_and_summarize_differences(directory)
summary_table

Parameter,libd_chr1-chr1_all-libd_chr1-chr1_all-1-200-dynamic-1corestotal-1corepera-20240319-124236-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-1001-1200-dynamic-1corestotal-1corepera-20240319-124337-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-1201-1400-dynamic-1corestotal-1corepera-20240319-124354-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-1401-1600-dynamic-1corestotal-1corepera-20240319-124357-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-1601-1800-dynamic-1corestotal-1corepera-20240319-124408-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-1801-2000-dynamic-1corestotal-1corepera-20240319-124418-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-2001-2200-dynamic-1corestotal-1corepera-20240319-124420-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-201-400-dynamic-1corestotal-1corepera-20240319-124251-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-2201-2400-dynamic-1corestotal-1corepera-20240319-124424-summary.csv,⋯,libd_chr1-chr1_all-libd_chr1-chr1_all-801-1000-dynamic-1corestotal-1corepera-20240319-124322-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-8201-8400-dynamic-1corestotal-1corepera-20240319-124926-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-8401-8600-dynamic-1corestotal-1corepera-20240319-124928-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-8601-8800-dynamic-1corestotal-1corepera-20240319-124947-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-8801-9000-dynamic-1corestotal-1corepera-20240319-125007-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-9001-9200-dynamic-1corestotal-1corepera-20240319-125026-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-9201-9400-dynamic-1corestotal-1corepera-20240319-125040-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-9401-9600-dynamic-1corestotal-1corepera-20240319-125047-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-9601-9800-dynamic-1corestotal-1corepera-20240319-125103-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-9801-10000-dynamic-1corestotal-1corepera-20240319-125107-summary.csv
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
chunk1,1,1001,1201,1401,1601,1801,2001,201,2201,⋯,801,8201,8401,8601,8801,9001,9201,9401,9601,9801
chunk2,200,1200,1400,1600,1800,2000,2200,400,2400,⋯,1000,8400,8600,8800,9000,9200,9400,9600,9800,10000
tag,libd_chr1-chr1_all-1-200-dynamic-1corestotal-1corepera-20240319-124236,libd_chr1-chr1_all-1001-1200-dynamic-1corestotal-1corepera-20240319-124337,libd_chr1-chr1_all-1201-1400-dynamic-1corestotal-1corepera-20240319-124354,libd_chr1-chr1_all-1401-1600-dynamic-1corestotal-1corepera-20240319-124357,libd_chr1-chr1_all-1601-1800-dynamic-1corestotal-1corepera-20240319-124408,libd_chr1-chr1_all-1801-2000-dynamic-1corestotal-1corepera-20240319-124418,libd_chr1-chr1_all-2001-2200-dynamic-1corestotal-1corepera-20240319-124420,libd_chr1-chr1_all-201-400-dynamic-1corestotal-1corepera-20240319-124251,libd_chr1-chr1_all-2201-2400-dynamic-1corestotal-1corepera-20240319-124424,⋯,libd_chr1-chr1_all-801-1000-dynamic-1corestotal-1corepera-20240319-124322,libd_chr1-chr1_all-8201-8400-dynamic-1corestotal-1corepera-20240319-124926,libd_chr1-chr1_all-8401-8600-dynamic-1corestotal-1corepera-20240319-124928,libd_chr1-chr1_all-8601-8800-dynamic-1corestotal-1corepera-20240319-124947,libd_chr1-chr1_all-8801-9000-dynamic-1corestotal-1corepera-20240319-125007,libd_chr1-chr1_all-9001-9200-dynamic-1corestotal-1corepera-20240319-125026,libd_chr1-chr1_all-9201-9400-dynamic-1corestotal-1corepera-20240319-125040,libd_chr1-chr1_all-9401-9600-dynamic-1corestotal-1corepera-20240319-125047,libd_chr1-chr1_all-9601-9800-dynamic-1corestotal-1corepera-20240319-125103,libd_chr1-chr1_all-9801-10000-dynamic-1corestotal-1corepera-20240319-125107
time_started,2024-03-19 12:42:49,2024-03-19 12:43:57,2024-03-19 12:44:11,2024-03-19 12:44:28,2024-03-19 12:44:28,2024-03-19 12:44:33,2024-03-19 12:44:26,2024-03-19 12:42:58,2024-03-19 12:44:36,⋯,2024-03-19 12:43:35,2024-03-19 17:23

## Compute core-hours per test

In [3]:
windows <- c(2000,4000,6000,8000,10000,20000,40000,50000,60000,80000,100000,150000,200000,300000,350000,400000,500000)
n_windows <- length(windows)
alphas <- 1 #seq(0, 1, .25) # left out 0 on first run
n_alphas <- length(alphas)
chunks <- 10000
n_tests <- n_windows * n_alphas * chunks
n_tests

[1] 170000

In [4]:
alphas

[1] 1

In [5]:
colnames(summary_table)

[1] "Parameter"                                                                                                 
 [2] "libd_chr1-chr1_all-libd_chr1-chr1_all-1-200-dynamic-1corestotal-1corepera-20240319-124236-summary.csv"     
 [3] "libd_chr1-chr1_all-libd_chr1-chr1_all-1001-1200-dynamic-1corestotal-1corepera-20240319-124337-summary.csv" 
 [4] "libd_chr1-chr1_all-libd_chr1-chr1_all-1201-1400-dynamic-1corestotal-1corepera-20240319-124354-summary.csv" 
 [5] "libd_chr1-chr1_all-libd_chr1-chr1_all-1401-1600-dynamic-1corestotal-1corepera-20240319-124357-summary.csv" 
 [6] "libd_chr1-chr1_all-libd_chr1-chr1_all-1601-1800-dynamic-1corestotal-1corepera-20240319-124408-summary.csv" 
 [7] "libd_chr1-chr1_all-libd_chr1-chr1_all-1801-2000-dynamic-1corestotal-1corepera-20240319-124418-summary.csv" 
 [8] "libd_chr1-chr1_all-libd_chr1-chr1_all-2001-2200-dynamic-1corestotal-1corepera-20240319-124420-summary.csv" 
 [9] "libd_chr1-chr1_all-libd_chr1-chr1_all-201-400-dynamic-1corestotal-1corepera-20240319-124251-summary.csv"   
[10] "libd_chr1-chr1_all-libd_chr1-chr1_all-2201-2400-dynamic-1corestotal-1corepera-20240319-124424-summary.csv" 
[11] "libd_chr1-chr1_all-libd_chr1-chr1_all-2401-2600-dynamic-1corestotal-1corepera-20240319-124440-summary.csv" 
[12] "libd_chr1-chr1_all-libd_chr1-chr1_all-2601-2800-dynamic-1corestotal-1corepera-20240319-124500-summary.csv" 
[13] "libd_chr1-chr1_all-libd_chr1-chr1_all-2801-3000-dynamic-1corestotal-1corepera-20240319-124519-summary.csv" 
[14] "libd_chr1-chr1_all-libd_chr1-chr1_all-3001-3200-dynamic-1corestotal-1corepera-20240319-124528-summary.csv" 
[15] "libd_chr1-chr1_all-libd_chr1-chr1_all-3201-3400-dynamic-1corestotal-1corepera-20240319-124532-summary.csv" 
[16] "libd_chr1-chr1_all-libd_chr1-chr1_all-3401-3600-dynamic-1corestotal-1corepera-20240319-124543-summary.csv" 
[17] "libd_chr1-chr1_all-libd_chr1-chr1_all-3601-3800-dynamic-1corestotal-1corepera-20240319-124603-summary.csv" 
[18] "libd_chr1-chr1_all-libd_chr1-chr1_all-3801-4000-dynamic-1corestotal-1corepera-20240319-124616-summary.csv" 
[19] "libd_chr1-chr1_all-libd_chr1-chr1_all-4001-4200-dynamic-1corestotal-1corepera-20240319-124627-summary.csv" 
[20] "libd_chr1-chr1_all-libd_chr1-chr1_all-401-600-dynamic-1corestotal-1corepera-20240319-124257-summary.csv"   
[21] "libd_chr1-chr1_all-libd_chr1-chr1_all-4201-4400-dynamic-1corestotal-1corepera-20240319-124629-summary.csv" 
[22] "libd_chr1-chr1_all-libd_chr1-chr1_all-4401-4600-dynamic-1corestotal-1corepera-20240319-124638-summary.csv" 
[23] "libd_chr1-chr1_all-libd_chr1-chr1_all-4601-4800-dynamic-1corestotal-1corepera-20240319-124642-summary.csv" 
[24] "libd_chr1-chr1_all-libd_chr1-chr1_all-4801-5000-dynamic-1corestotal-1corepera-20240320-125744-summary.csv" 
[25] "libd_chr1-chr1_all-libd_chr1-chr1_all-5001-5200-dynamic-1corestotal-1corepera-20240320-125805-summary.csv" 
[26] "libd_chr1-chr1_all-libd_chr1-chr1_all-5201-5400-dynamic-1corestotal-1corepera-20240319-124700-summary.csv" 
[27] "libd_chr1-chr1_all-libd_chr1-chr1_all-5401-5600-dynamic-1corestotal-1corepera-20240319-124711-summary.csv" 
[28] "libd_chr1-chr1_all-libd_chr1-chr1_all-5601-5800-dynamic-1corestotal-1corepera-20240319-124725-summary.csv" 
[29] "libd_chr1-chr1_all-libd_chr1-chr1_all-5801-6000-dynamic-1corestotal-1corepera-20240319-124735-summary.csv" 
[30] "libd_chr1-chr1_all-libd_chr1-chr1_all-6001-6200-dynamic-1corestotal-1corepera-20240319-124736-summary.csv" 
[31] "libd_chr1-chr1_all-libd_chr1-chr1_all-601-800-dynamic-1corestotal-1corepera-20240319-124318-summary.csv"   
[32] "libd_chr1-chr1_all-libd_chr1-chr1_all-6201-6400-dynamic-1corestotal-1corepera-20240319-124738-summary.csv" 
[33] "libd_chr1-chr1_all-libd_chr1-chr1_all-6401-6600-dynamic-1corestotal-1corepera-20240319-124747-summary.csv" 
[34] "libd_chr1-chr1_all-libd_chr1-chr1_all-6601-6800-dynamic-1corestotal-1corepera-20240319-124752-summary.csv" 
[35] "libd_chr1-chr1_all-libd_chr1-chr1_all-6801-7000-dynamic-1corestotal-1corepera-20240319-124808-summary.csv" 
[36] "libd_

In [6]:
summary_table

Parameter,libd_chr1-chr1_all-libd_chr1-chr1_all-1-200-dynamic-1corestotal-1corepera-20240319-124236-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-1001-1200-dynamic-1corestotal-1corepera-20240319-124337-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-1201-1400-dynamic-1corestotal-1corepera-20240319-124354-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-1401-1600-dynamic-1corestotal-1corepera-20240319-124357-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-1601-1800-dynamic-1corestotal-1corepera-20240319-124408-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-1801-2000-dynamic-1corestotal-1corepera-20240319-124418-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-2001-2200-dynamic-1corestotal-1corepera-20240319-124420-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-201-400-dynamic-1corestotal-1corepera-20240319-124251-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-2201-2400-dynamic-1corestotal-1corepera-20240319-124424-summary.csv,⋯,libd_chr1-chr1_all-libd_chr1-chr1_all-801-1000-dynamic-1corestotal-1corepera-20240319-124322-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-8201-8400-dynamic-1corestotal-1corepera-20240319-124926-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-8401-8600-dynamic-1corestotal-1corepera-20240319-124928-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-8601-8800-dynamic-1corestotal-1corepera-20240319-124947-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-8801-9000-dynamic-1corestotal-1corepera-20240319-125007-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-9001-9200-dynamic-1corestotal-1corepera-20240319-125026-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-9201-9400-dynamic-1corestotal-1corepera-20240319-125040-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-9401-9600-dynamic-1corestotal-1corepera-20240319-125047-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-9601-9800-dynamic-1corestotal-1corepera-20240319-125103-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-9801-10000-dynamic-1corestotal-1corepera-20240319-125107-summary.csv
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
chunk1,1,1001,1201,1401,1601,1801,2001,201,2201,⋯,801,8201,8401,8601,8801,9001,9201,9401,9601,9801
chunk2,200,1200,1400,1600,1800,2000,2200,400,2400,⋯,1000,8400,8600,8800,9000,9200,9400,9600,9800,10000
tag,libd_chr1-chr1_all-1-200-dynamic-1corestotal-1corepera-20240319-124236,libd_chr1-chr1_all-1001-1200-dynamic-1corestotal-1corepera-20240319-124337,libd_chr1-chr1_all-1201-1400-dynamic-1corestotal-1corepera-20240319-124354,libd_chr1-chr1_all-1401-1600-dynamic-1corestotal-1corepera-20240319-124357,libd_chr1-chr1_all-1601-1800-dynamic-1corestotal-1corepera-20240319-124408,libd_chr1-chr1_all-1801-2000-dynamic-1corestotal-1corepera-20240319-124418,libd_chr1-chr1_all-2001-2200-dynamic-1corestotal-1corepera-20240319-124420,libd_chr1-chr1_all-201-400-dynamic-1corestotal-1corepera-20240319-124251,libd_chr1-chr1_all-2201-2400-dynamic-1corestotal-1corepera-20240319-124424,⋯,libd_chr1-chr1_all-801-1000-dynamic-1corestotal-1corepera-20240319-124322,libd_chr1-chr1_all-8201-8400-dynamic-1corestotal-1corepera-20240319-124926,libd_chr1-chr1_all-8401-8600-dynamic-1corestotal-1corepera-20240319-124928,libd_chr1-chr1_all-8601-8800-dynamic-1corestotal-1corepera-20240319-124947,libd_chr1-chr1_all-8801-9000-dynamic-1corestotal-1corepera-20240319-125007,libd_chr1-chr1_all-9001-9200-dynamic-1corestotal-1corepera-20240319-125026,libd_chr1-chr1_all-9201-9400-dynamic-1corestotal-1corepera-20240319-125040,libd_chr1-chr1_all-9401-9600-dynamic-1corestotal-1corepera-20240319-125047,libd_chr1-chr1_all-9601-9800-dynamic-1corestotal-1corepera-20240319-125103,libd_chr1-chr1_all-9801-10000-dynamic-1corestotal-1corepera-20240319-125107
time_started,2024-03-19 12:42:49,2024-03-19 12:43:57,2024-03-19 12:44:11,2024-03-19 12:44:28,2024-03-19 12:44:28,2024-03-19 12:44:33,2024-03-19 12:44:26,2024-03-19 12:42:58,2024-03-19 12:44:36,⋯,2024-03-19 12:43:35,2024-03-19 17:23

In [7]:
summary_table <- as.data.frame(t(summary_table))
colnames(summary_table) <- summary_table[1, ]
summary_table <- summary_table[-1, ]
summary_table$number_cores <- 1

In [8]:
summary_table$runtime_seconds <- sapply(summary_table$runtime, function(x) {
  parts <- strsplit(x, ":")[[1]]
  as.numeric(parts[1]) * 3600 + as.numeric(parts[2]) * 60 + as.numeric(parts[3])
})

In [9]:
summary_table$runtime_hours <- summary_table$runtime_seconds/3600

In [10]:
summary_table$runtime_core_hours <- summary_table$runtime_hours * summary_table$number_cores
summary_table$runtime_core_seconds <- summary_table$runtime_seconds * summary_table$number_cores
summary_table$core_hours_per_test <- summary_table$runtime_core_hours / n_tests
summary_table$core_seconds_per_test <- summary_table$runtime_core_seconds / n_tests

In [11]:
summary_table

,chunk1,chunk2,tag,time_started,time_finished,runtime,type_CPU,amount_RAM,physical_cores,scaffold_ID,number_cores,runtime_seconds,runtime_hours,runtime_core_hours,runtime_core_seconds,core_hours_per_test,core_seconds_per_test
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
libd_chr1-chr1_all-libd_chr1-chr1_all-1-200-dynamic-1corestotal-1corepera-20240319-124236-summary.csv,1,200,libd_chr1-chr1_all-1-200-dynamic-1corestotal-1corepera-20240319-124236,2024-03-19 12:42:49,2024-03-19 17:56:59,05:14:10,AMD Opteron(TM) Processor 6234,0.492048908025026,24,libd_chr1-chr1_all-libd_chr1-chr1_all-1-200-dynamic-1corestotal-1corepera-20240319-124236,1,18850,5.236111,5.236111,18850,3.080065e-05,0.11088235
libd_chr1-chr1_all-libd_chr1-chr1_all-1001-1200-dynamic-1corestotal-1corepera-20240319-124337-summary.csv,1001,1200,libd_chr1-chr1_all-1001-1200-dynamic-1corestotal-1corepera-20240319-124337,2024-03-19 12:43:57,2024-03-19 15:44:28,03:00:30,Intel(R) Xeon(R) CPU E5-2650L v3 @ 1.80GHz,0.122832637280226,24,libd_chr1-chr1_all-libd_chr1-chr1_all-1001-1200-dynamic-1corestotal-1corepera-20240319-124337,1,10830,3.008333,3.008333,10830,1.769608e-05,0.06370588
libd_chr1-chr1_all-libd_chr1-chr1_all-1201-1400-dynamic-1corestotal-1corepera-20240319-124354-summary.csv,1201,1400,libd_chr1-chr1_all-1201-1400-dynamic-1corestotal-1corepera-20240319-124354,2024-03-19 12:44:11,2024-03-19 15:58:53,03:14:42,Intel(R) Xeon(R) CPU E5-2650L v3 @ 1.80GHz,0.122832637280226,24,libd_chr1-chr1_all-libd_chr1-chr1_all-1201-1400-dynamic-1corestotal-1corepera-20240319-124354,1,11682,3.245000,3.245000,11682,1.908824e-05,0.06871765
libd_chr1-chr1_all-libd_chr1-chr1_all-1401-1600-dynamic-1corestotal-1corepera-20240319-124357-summary.csv,1401,1600,libd_chr1-chr1_all-1401-1600-dynamic-1corestotal-1corepera-20240319-124357,2024-03-19 12:44:28,2024-03-19 20:00:01,07:15:32,AMD Opteron(TM) Processor 6234,0.476668059825897,24,libd_chr1-chr1_all-libd_chr1-chr1_all-1401-1600-dynamic-1corestotal-1corepera-20240319-124357,1,26132,7.258889,7.258889,26132,4.269935e-05,0.15371765
libd_chr1-chr1_all-libd_chr1-chr1_all-1601-1800-dynamic-1corestotal-1corepera-20240319-124408-summary.csv,1601,1800,libd_chr1-chr1_all-1601-1800-dynamic-1corestotal-1corepera-20240319-124408,2024-03-19 12:44:28,2024-03-19 19:58:14,07:13:45,AMD Opteron(TM) Processor 6234,0.476668059825897,24,libd_chr1-chr1_all-libd_chr1-chr1_all-1601-1800-dynamic-1corestotal-1corepera-20240319-124408,1,26025,7.229167,7.229167,26025,4.252451e-05,0.15308824
libd_chr1-chr1_all-libd_chr1-chr1_all-1801-2000-dynamic-1corestotal-1corepera-20240319-124418-summary.csv,1801,2000,libd_chr1-chr1_all-1801-2000-dynamic-1corestotal-1corepera-20240319-124418,2024-03-19 12:44:33,2024-03-19 19:48:45,07:04:12,AMD Opteron(TM) Processor 6234,0.476668059825897,24,libd_chr1-chr1_all-libd_chr1-chr1_all-1801-2000-dynamic-1corestotal-1corepera-20240319-124418,1,25452,7.070000,7.070000,25452,4.158824e-05,0.14971765
libd_chr1-chr1_all-libd_chr1-chr1_all-2001-2200-dynamic-1corestotal-1corepera-20240319-124420-summary.csv,2001,2200,libd_chr1-chr1_all-2001-2200-dynamic-1corestotal-1corepera-20240319-124420,2024-03-19 12:44:26,2024-03-19 15:58:56,03:14:29,Intel(R) Xeon(R) CPU E5-2650L v3 @ 1.80GHz,0.122832637280226,24,libd_chr1-chr1_all-libd_chr1-chr1_all-2001-2200-dynamic-1corestotal-1corepera-20240319-124420,1,11669,3.241389,3.241389,11669,1.906699e-05,0.06864118
libd_chr1-chr1_all-libd_chr1-chr1_all-201-400-dynamic-1corestotal-1corepera-20240319-124251-summary.csv,201,400,libd_chr1-chr1_all-201-400-dynamic-1corestotal-1corepera-20240319-124251,2024-03-19 12:42:58,2024-03-19 17:56:57,05:13:58,AMD Opteron(TM) Processor 6234,0.492048908025026,24,libd_chr1-chr1_all-libd_chr1-chr1_all-201-400-dynamic-1corestotal-1corepera-20240319-124251,1,18838,5.232778,5.232778,18838,3.078105e-05,0.11081176
libd_chr1-chr1_all-libd_chr1-chr1_all-2201-2400-dynamic-1corestotal-1corepera-20240319-124424-summary.csv,2201,2400,libd_ch

In [12]:
#summary_table[which.min(summary_table$core_hours_per_test), ]

In [13]:
data.table::fwrite(summary_table, "05-OUT_SLURM_benchmarks_a2.9.csv")

## How long for whole dataset?

In [14]:
n_sites <- 26000000

In [15]:
n_sites*0.0001107989 # From 1000000-1001000

[1] 2880.771

In [16]:
n_sites*mean(summary_table$core_hours_per_test)

[1] 660.5759

In [17]:
# 1240